In [1]:
import os
import re
import string
import unicodedata
import pandas as pd 
import itertools
import numpy as np
from symspellpy.symspellpy import SymSpell, Verbosity
import pkg_resources
from spellchecker import SpellChecker
import html
import csv

In [9]:
df = pd.read_csv('train.csv')

## hashtags

In [10]:
def spell_check_words(word):
    # lookup suggestions for multi word string input
    spell = SymSpell(max_dictionary_edit_distance=3, prefix_length=7)
    dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
    bigram_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_bigramdictionary_en_243_342.txt")
    spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
    spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)
    suggestions = spell.lookup_compound(input_term, max_edit_distance=2)
# display suggestion term, edit distance, and term frequency
    for suggestion in suggestions:
        print(suggestion)   

In [11]:
def spell_check_word(word):
    # lookup suggestions for individual words
    spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=5)
    dictionary_path = pkg_resources.resource_filename(
                      "symspellpy", "frequency_dictionary_en_82_765.txt")
    spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
    corrections = spell.lookup(word, Verbosity.TOP)
    for correction in corrections:
        print(correction)

In [50]:
# functions related to expanded hashtags in text
def pascal_case_split(text):
    # expand hashtags formatted in pascal case, ex: #ThisIsPascal
    return re.sub(r'([A-Z])([?=a-z0-9+])', r' \1\2', text)

def camel_case_split(text):
    # expand hashtags formatted in pascal case, ex: #thisIsCamel
    return re.sub(r'([a-z0-9+])([?<=A-Z])', r'\1 \2', text)

def remove_hash(text):
    # remove hash symbol in front of hashtag text and remove non unicode chars
    return re.sub('#', '', text)


def remove_non_alpha(text):
    # function to remove non unicode characters from string
    return re.sub('[^a-zA-Z_, ]', '', text)

        
def expand_hashtags(text):
    # combine hashtag split functions for specific cases and remove hash
    c_text = camel_case_split(text)
    p_text = pascal_case_split(c_text)
    lu_text = desegment_hashtag(p_text)
    h_text = remove_hash(lu_text)
    return html.unescape(h_text)

def desegment_hashtag(text):
    spell = SymSpell(max_dictionary_edit_distance=0, prefix_length=7)
    dictionary_path = pkg_resources.resource_filename("symspellpy","frequency_dictionary_en_82_765.txt")
    spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
    if text.isupper() | text.islower() and len(text)>= 15:
        result = spell.word_segmentation(text)
        text = result.corrected_string
    return text

def monocase_mask(text):
    # returns boolean mask for monocase strings 
    if text.isupper() | text.islower() and text != 'nan':
        return True

    else:
        return False
    
def mono_check(text):
    # returns boolean mask for monocase strings 
    text.isupper() | text.islower() and text != 'nan'


In [ ]:
# create column for extracted hashtags
df['hashtags'] = df.text.str.findall(r'(?:(?<=\s)|(?<=^))#.*?(?=\s|$)')
df.hashtags = df.hashtags.astype(str)

In [ ]:
expanded_hashtags = df.hashtags.apply(lambda x: (expand_hashtags(x)))
df['expanded_hashtags'] = expanded_hashtags.str.strip()